<a href="https://colab.research.google.com/github/StuSe/data-science-and-big-data-analytics/blob/master/Copy_of_Untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.exceptions import NotFittedError
import tensorflow as tf
from tqdm.notebook import tqdm, trange
import keras.backend as K

In [0]:
print("Loading data")
file = './train.csv'
data = pd.read_csv(file)
sub = pd.read_csv('./test.csv')
print("Preprocessing")

Loading data
Preprocessing


In [0]:
def preprocess_OHE(data_train, data_sub):
    X = data_train['Sequence']
    Y = data_train['Active']
    X_sub = data_sub['Sequence']
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.01, random_state=42)

    enc = OneHotEncoder()
    X_train = preprocess_OHE_X(X_train, enc)
    X_test = preprocess_OHE_X(X_test, enc)
    X_sub = preprocess_OHE_X(X_sub, enc)
    return X_train, X_test, Y_train, Y_test, X_sub

def preprocess_OHE_X(X_i, enc):
    X_i = X_i.apply(lambda x: list(x))
    X_i = np.array(X_i.tolist())
    X_i[X_i=='B'] = 'D'
    
    try:
        X_i = enc.transform(X_i).toarray()
    except NotFittedError:
        enc.fit(X_i)
        X_i = enc.transform(X_i).toarray()
    return X_i

X_train, X_test, Y_train, Y_test, X_sub = preprocess_OHE(data, sub)

In [0]:
print("Training model")
model = tf.keras.Sequential([tf.keras.layers.Dense(60, input_shape=X_train[0].shape, activation='relu'),
                             tf.keras.layers.Dense(20, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

class_weight = {0: 1.,
                1: 50.}

"""def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val"""

loss_function = tf.keras.losses.BinaryCrossentropy()
test_accuracy = f1_score


def binary_step(x):
    x[x<0.5] = 0.
    x[x>=0.5] = 1.
    return x


def test_step(data, labels):
    predictions = model.predict(data)
    predictions = binary_step(predictions[:,0])
    print(f"f1: {test_accuracy(labels, predictions)}")

model.summary()

Training model
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 60)                4860      
_________________________________________________________________
dense_13 (Dense)             (None, 20)                1220      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 21        
Total params: 6,101
Trainable params: 6,101
Non-trainable params: 0
_________________________________________________________________


In [0]:
EPOCHS = 100

def partition_data(items, n):
    res = []
    for it in items:
      res.append(np.array_split(it, n,axis=0))
    return list(map(list, zip(*res)))

#train_ds = partition_data([X_train, Y_train], 5)

model.compile(optimizer, loss_function)
for epoch in trange(EPOCHS):
    print(f"Epoch {epoch+1}")
    # Reset the metrics at the start of the next epoch
    #for data, labels in train_ds:
    #    model.fit(data, labels, class_weight=class_weight)
    model.fit(X_train, Y_train, class_weight=class_weight)
    test_step(X_test, Y_test)



Epoch 1
3465/3465 [==============================] - 5s 1ms/step - loss: 1.1059
f1: 0.37130801687763715
Epoch 2
3465/3465 [==============================] - 4s 1ms/step - loss: 0.4890
f1: 0.4512820512820513
Epoch 3
3465/3465 [==============================] - 5s 1ms/step - loss: 0.4263
f1: 0.4591836734693877
Epoch 4
3465/3465 [==============================] - 5s 1ms/step - loss: 0.3898
f1: 0.4712041884816754
Epoch 5
3465/3465 [==============================] - 5s 1ms/step - loss: 0.3549
f1: 0.4972375690607735
Epoch 6
3465/3465 [==============================] - 5s 1ms/step - loss: 0.3216
f1: 0.5263157894736842
Epoch 7
3465/3465 [==============================] - 4s 1ms/step - loss: 0.2908
f1: 0.5325443786982248
Epoch 8
3465/3465 [==============================] - 5s 1ms/step - loss: 0.2627
f1: 0.6
Epoch 9
3465/3465 [==============================] - 5s 1ms/step - loss: 0.2389
f1: 0.6122448979591837
Epoch 10
3465/3465 [==============================] - 5s 1ms/step - loss: 0.2184
f1: 0.

In [0]:
print("Predicting on submission data")
Y_sub = model.predict(X_sub)
Y_sub = binary_step(Y_sub[:,0])
np.savetxt('./submit.csv', Y_sub, fmt='%d')

Predicting on submission data
